In [20]:
import mne
import os
import numpy as np
import pandas as pd

from mne.time_frequency import tfr_multitaper, tfr_morlet, psd_array_welch
#from mne_features.univariate import compute_spect_entropy
from scipy.stats import kurtosis, skew

Read file


In [21]:
path_file = '/Users/sobieddch/UdG-TFM/ds004504/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set'
raw = mne.io.read_raw_eeglab(path_file, preload=False, verbose='CRITICAL')
raw

<RawEEGLAB | sub-002_task-eyesclosed_eeg.set, 19 x 396550 (793.1 s), ~57.5 MB, data loaded>

Epochs


In [22]:
sfreq = raw.info['sfreq']

start_time = 0
end_time = 600
duration = 60.0
overlapping = 30.0

epochs = mne.make_fixed_length_epochs(raw.copy().crop(tmin=start_time, tmax=end_time), duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL')
epochs

Number of events,19
Events,1: 19
Time range,0.000 – 59.998 s
Baseline,off


In [42]:
frequency_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

def get_Features_PSD(raw, type='epoch', channels=None, start_time = 0, end_time = 600, duration = 60.0, overlapping = 30.0):
    # variables
    sfreq = raw.info['sfreq']
    
    # Select channels
    if channels==None: ch = raw.ch_names
    else: ch = channels        
    
    if type=='epoch':
        # Get Epochs Data
        data = mne.make_fixed_length_epochs(raw.copy().pick(ch).crop(tmin=start_time, tmax=end_time),
                                        duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL').get_data()
    elif type=='raw':
        data = raw.copy().pick(ch).crop(tmin=start_time, tmax=end_time).get_data()
    
    # Compute features
    results = {}
    for band, (fmin,fmax) in frequency_bands.items():
        # Apply bandpass filter to select only the frequency band range
        data_filtered = mne.filter.filter_data(data, sfreq=sfreq, l_freq=fmin, h_freq=fmax, verbose='CRITICAL')
        # Compute the PSD using Welch's method
        psd_all, freqs_all = psd_array_welch(data, sfreq=sfreq, verbose='CRITICAL')
        psd, freqs = psd_array_welch(data_filtered, sfreq=sfreq, verbose='CRITICAL')
        
        # Compute metrics
        total_power = psd.sum()
        relative_power = total_power / psd_all.sum()
        average_power = psd.mean()
        #entropy = compute_spect_entropy(data_filtered, sfreq=sfreq)
        entropy = -np.sum(np.log(psd) * psd)
        std_dev = np.std(data_filtered)
        peak_to_peak = np.ptp(data_filtered)
        kurt = kurtosis(data_filtered)
        skewness = skew(data_filtered)
        # Store results
        results[f'{band}_total_power'] = total_power
        results[f'{band}_relative_power'] = relative_power
        results[f'{band}_average_power'] = average_power
        results[f'{band}_spectral_entropy'] = entropy
        results[f'{band}_peak_to_peak'] = peak_to_peak
        results[f'{band}_std_dev'] = std_dev
        results[f'{band}_kurtosis'] = np.mean(kurt)
        results[f'{band}_skewness'] = np.mean(skewness)
        
    return results

In [24]:
results_fp1 = get_Features_PSD(raw, channels=['Fp1'])
results_all = get_Features_PSD(raw)

In [25]:
path_file = '/Users/sobieddch/UdG-TFM/ds004504/derivatives/sub-067/eeg/sub-067_task-eyesclosed_eeg.set'
raw2 = mne.io.read_raw_eeglab(path_file, preload=False, verbose='CRITICAL')
raw2

<RawEEGLAB | sub-067_task-eyesclosed_eeg.set, 19 x 321550 (643.1 s), ~46.6 MB, data loaded>

In [26]:
results2_fp1 = get_Features_PSD(raw2, channels=['Fp1'])
results2_all = get_Features_PSD(raw2)

Comparison


In [27]:
print(results_fp1['alpha'])
print(results2_fp1['alpha'])

{'total_power': 3.336569476807225e-10, 'relative_power': 0.1075597516961631, 'average_power': 1.3613094560616993e-13, 'spectral_entropy': 8.70257954556739e-09, 'std': 5.836209307805536e-06, 'peak_to_peak': 5.908582730364675e-05, 'kurtosis': -0.12507998247860266, 'skewness': -6.159269075602042e-05}
{'total_power': 4.691713946889086e-10, 'relative_power': 0.08507585632630149, 'average_power': 1.9142039766989334e-13, 'spectral_entropy': 1.219010026955677e-08, 'std': 6.941888370332027e-06, 'peak_to_peak': 0.00011451900992373648, 'kurtosis': 0.07730005505862288, 'skewness': -0.00019421167907372533}


---


## TFR


In [28]:
tfr = tfr_multitaper(epochs.copy(), freqs=np.arange(frequency_bands['theta'][0], frequency_bands['theta'][1], 1), n_cycles=7, return_itc=False, verbose='critical')

In [29]:
tfr.data

array([[[5.20698949e-09, 5.24066393e-09, 5.27411406e-09, ...,
         6.03026768e-09, 6.01427671e-09, 5.99884629e-09],
        [3.52050504e-09, 3.52319860e-09, 3.52562392e-09, ...,
         4.42067768e-09, 4.41254591e-09, 4.40435942e-09],
        [2.59382490e-09, 2.61262182e-09, 2.63176692e-09, ...,
         2.98694801e-09, 2.97899320e-09, 2.97193919e-09],
        [2.67006930e-09, 2.68037920e-09, 2.68963295e-09, ...,
         3.57567104e-09, 3.56461686e-09, 3.55326384e-09]],

       [[5.24955276e-09, 5.28237725e-09, 5.31444330e-09, ...,
         5.75298698e-09, 5.74288567e-09, 5.73259496e-09],
        [3.27255802e-09, 3.28006274e-09, 3.28705244e-09, ...,
         4.01483521e-09, 4.01295600e-09, 4.01029004e-09],
        [2.65460762e-09, 2.67982437e-09, 2.70521113e-09, ...,
         2.96276693e-09, 2.95819807e-09, 2.95349325e-09],
        [2.84607309e-09, 2.85661191e-09, 2.86663485e-09, ...,
         3.79024311e-09, 3.77729688e-09, 3.76373861e-09]],

       [[5.29957445e-09, 5.33247728e

In [30]:
tfr.data.sum()

0.014982802228134324

In [31]:
#np.mean(tfr.data)
tfr.data.mean()

6.571404486023826e-09

In [32]:
kurtosis(tfr.data, axis=(1,2))

array([ 0.23293396,  0.06126115,  0.06579434,  0.10804024, -0.11344009,
       -0.2540888 ,  0.48077593,  0.51818699, -0.73576247, -0.59753481,
        0.29575798,  0.03374003,  0.19627011, -0.10029733, -0.40640983,
       -0.52679684,  0.08685125, -0.19793916,  0.53189139])

In [33]:
def get_Features_TFR(raw, channels=None, start_time = 0, end_time = 600, duration = 60.0, overlapping = 30.0, n_cycles=7, time_bandwidth=4.0):
    # variables
    sfreq = raw.info['sfreq']
    
    # Select channels
    if channels==None: ch = raw.ch_names
    else: ch = channels        
    
    # Get Epochs Data
    epochs = mne.make_fixed_length_epochs(raw.copy().pick(ch).crop(tmin=start_time, tmax=end_time),
                                    duration=duration, overlap=overlapping, preload=True, verbose='CRITICAL')
    
    # Compute features
    results = {}
    for band, (fmin,fmax) in frequency_bands.items():
        # Get the TFR object
        tfr = tfr_multitaper(epochs.copy().pick(ch), freqs=np.arange(fmin,fmax,1), n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=False, verbose='critical')
        # Compute features
        total_power = tfr.data.sum()
        average_power = tfr.data.mean()
        peak_power = tfr.data.max()
        std_dev = tfr.data.std()
        skewness = skew(tfr.data, axis=(1,2))
        kurt = kurtosis(tfr.data, axis=(1,2))
        # Store results
        results[f'{band}_total_power'] = total_power
        results[f'{band}_average_power'] = average_power
        results[f'{band}_peak_power'] = peak_power
        results[f'{band}_std'] = std_dev
        results[f'{band}_kurtosis'] = np.mean(kurt)
        results[f'{band}_skewness'] = np.mean(skewness)
    
    return results

In [34]:
tfr_results = get_Features_TFR(raw, channels=['Fp1'])
tfr_results

{'delta_total_power': 0.019381931071842277,
 'delta_average_power': 1.615160922653523e-07,
 'delta_peak_power': 6.714592259707862e-07,
 'delta_std': 2.17037647797898e-07,
 'delta_kurtosis': -0.32108539356355337,
 'delta_skewness': 1.2244486095742626,
 'theta_total_power': 0.0007447824105880776,
 'theta_average_power': 6.20652008823398e-09,
 'theta_peak_power': 1.3716200970207192e-08,
 'theta_std': 2.2186729367690723e-09,
 'theta_kurtosis': 0.23293396320498072,
 'theta_skewness': 1.0537821447068398,
 'alpha_total_power': 0.00044341224423544064,
 'alpha_average_power': 2.956081628236271e-09,
 'alpha_peak_power': 6.748293192793814e-09,
 'alpha_std': 1.2809305490327442e-09,
 'alpha_kurtosis': -0.815683144448466,
 'alpha_skewness': 0.19861260693369628,
 'beta_total_power': 0.00024643569885988926,
 'beta_average_power': 4.832072526664496e-10,
 'beta_peak_power': 3.6124994686155763e-09,
 'beta_std': 2.6191528461941027e-10,
 'beta_kurtosis': 18.53449855618216,
 'beta_skewness': 2.7475910136641

---


In [37]:
data = raw.copy().crop(tmax=600).get_data()

In [38]:
psd, freqs = psd_array_welch(data, sfreq=sfreq, verbose='CRITICAL')

In [44]:
kurtosis(psd)

array([-0.43524605, -1.22704852, -0.34597562, -1.19436148, -0.4684757 ,
       -0.48001451, -0.83183038, -0.70868684, -0.73565021, -0.2558022 ,
        0.16459479, -0.38987066, -0.48608738, -0.6061629 , -1.13196806,
       -1.03380786, -0.47566682, -0.41573747, -0.22425904,  1.20308915,
        0.86114049,  0.3867365 , -0.56622267, -0.98344386, -0.91179387,
       -0.08929987, -0.14844746, -0.16298309, -0.18246912, -0.19388207,
       -0.21433213, -0.22478053, -0.23900007, -0.25071721, -0.26017896,
       -0.26909794, -0.27668836, -0.28491191, -0.29248023, -0.2956153 ,
       -0.30243026, -0.30680478, -0.31283877, -0.31478572, -0.32004661,
       -0.32325903, -0.3254504 , -0.33041231, -0.33061891, -0.33541048,
       -0.33642342, -0.33816182, -0.34097441, -0.34317079, -0.34414777,
       -0.34562642, -0.34891942, -0.34783523, -0.35084121, -0.35209096,
       -0.35324545, -0.35365454, -0.3557254 , -0.35602412, -0.35663428,
       -0.36001804, -0.35773381, -0.36024386, -0.36090513, -0.36

In [43]:
results_raw = get_Features_PSD(raw, type='raw', channels='Fp1')
results_raw

/var/folders/42/zkw31xg11x1f6dpl9zz6hmz00000gn/T/ipykernel_10625/4291331822.py:41: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt = kurtosis(data_filtered)
/var/folders/42/zkw31xg11x1f6dpl9zz6hmz00000gn/T/ipykernel_10625/4291331822.py:42: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness = skew(data_filtered)


{'delta_total_power': 1.2680716393320296e-10,
 'delta_relative_power': 0.7691802637456171,
 'delta_average_power': 9.83001270800023e-13,
 'delta_spectral_entropy': 2.9987984827081818e-09,
 'delta_peak_to_peak': 0.00022221169439472493,
 'delta_std_dev': 2.802584664248585e-05,
 'delta_kurtosis': nan,
 'delta_skewness': nan,
 'theta_total_power': 3.5319083189417285e-11,
 'theta_relative_power': 0.21423664783797036,
 'theta_average_power': 2.7379134255362234e-13,
 'theta_spectral_entropy': 9.036236645176576e-10,
 'theta_peak_to_peak': 7.729054987174171e-05,
 'theta_std_dev': 8.18523237083065e-06,
 'theta_kurtosis': nan,
 'theta_skewness': nan,
 'alpha_total_power': 1.74562592402894e-11,
 'alpha_relative_power': 0.10588526444397486,
 'alpha_average_power': 1.3531983907201087e-13,
 'alpha_spectral_entropy': 4.5599650804801075e-10,
 'alpha_peak_to_peak': 5.908582730364674e-05,
 'alpha_std_dev': 5.819140781898279e-06,
 'alpha_kurtosis': nan,
 'alpha_skewness': nan,
 'beta_total_power': 8.61726